# Training Image Generation

A training data set is generated here using
- 58 Backgrounds
- 3 Perspectives of each of the 6 pills

## Setup

In [1]:
# Import necessary packages
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
from random import seed
from random import random
from datetime import datetime

In [2]:
# set random number generator seed
seed(datetime.now())
!mkdir TrainingData

Ein Unterverzeichnis oder eine Datei mit dem Namen "TrainingData" existiert bereits.


In [3]:
# load pill images from disk
pills = []
for i in range(0, 18):
    pills.append(Image.open('AlphaPills/' + str(i) + '.png'));

In [4]:
# load background images
backgrounds = []
for i in range(0, 58):
    backgrounds.append(Image.open('Backgrounds/' + str(i) + '.jpg'));

## Dataset Generation
### Create training images with only one pill each

In [5]:
# pill_index: index of pill in image
# background_index: index of backgorund used in image
# name: file name of final image
def createTestImages(pill_index, background_index, name):
    # Determine pill images width and heigth
    pillWidth, pillHeight = pills[pill_index].size
    
    # Scale image by factor between 0.2 and 0.5
    factor = (0.5 * random() + 0.2)
    pillImage = pills[pill_index].resize((int(pillWidth * factor), int(pillHeight * factor)), Image.ANTIALIAS)
    
    # Rotate image by random degree
    rotation = 360.0 * random()
    pillImage = pillImage.rotate(rotation, expand=True)
    
    # recalculate new bounding box
    alpha = pillImage.getchannel('A')
    bbox = alpha.getbbox()
    pillImage = pillImage.crop(bbox)
    
    # determine new widht and heigth for pill image and background image
    pillWidth, pillHeight = pillImage.size
    backWidth, backHeight = backgrounds[background_index].size

    # determine random position for pill
    trainImage = backgrounds[background_index].copy()
    x = random() * (backWidth - pillWidth)
    y = random() * (backHeight - pillHeight)
    
    # render pill on background
    trainImage.paste(pillImage, (int(x), int(y)), pillImage)
    trainImage = trainImage.resize((1280, 962),Image.ANTIALIAS)
    
    # add random illumination
    enhancer = ImageEnhance.Brightness(trainImage)
    factor = 0.5 + random()
    trainImage = enhancer.enhance(factor)
    
    # save image
    trainImage.save('TrainingData/' + name + ".jpg", quality=95)
    
    # Determine Category
    # As each image has three perspectives, the pill can be calculated from image index
    category = -1;
    if pill_index < 3:
        category = 2
    elif pill_index < 6:
        category = 3
    elif pill_index < 9:
        category = 5
    elif pill_index < 12:
        category = 4
    elif pill_index < 15:
        category = 0
    elif pill_index < 18:
        category = 1
    
    # create descriptive txt file
    file = open('TrainingData/' + name + ".txt", "w")
    file.write(str(category) + " " + str((x + pillWidth/2)/backWidth) +" " + str((y + pillHeight/2)/backHeight) + " " + str(pillWidth/backWidth) + " " + str(pillHeight/backHeight)) 
    file.close()

### Create training images with multiple pills 

In [16]:
# index: index of final image, used in name and to determine background
def createTestImagesMultPills(index):
    name = "TestImage_" + str(index)
    trainImage = backgrounds[index%58].copy()
    file = open('TrainingData/' + name + ".txt", "w")

    # add random pills to background, each pill perspective has 20% if being rendered
    for pill_index in range(0, 18):
        if random() > 0.8:
            # Determine pill images width and heigth
            pillWidth, pillHeight = pills[pill_index].size

            # Scale image by factor between 0.2 and 0.5
            factor = (0.5 * random() + 0.2)
            pillImage = pills[pill_index].resize((int(pillWidth * factor), int(pillHeight * factor)), Image.ANTIALIAS)

            # Rotate image by random degree
            rotation = 360.0 * random()
            pillImage = pillImage.rotate(rotation, expand=True)

            # recalculate new bounding box
            alpha = pillImage.getchannel('A')
            bbox = alpha.getbbox()
            pillImage = pillImage.crop(bbox)

            # determine new widht and heigth for pill image and background image
            pillWidth, pillHeight = pillImage.size
            backWidth, backHeight = backgrounds[index%58].size

            # determine random position for pill
            x = random() * (backWidth - pillWidth)
            y = random() * (backHeight - pillHeight)

            # render pill on background
            trainImage.paste(pillImage, (int(x), int(y)), pillImage)

            # Determine Category
            # As each image has three perspectives, the pill can be calculated from image index
            category = -1;
            if pill_index < 3:
                category = 2
            elif pill_index < 6:
                category = 3
            elif pill_index < 9:
                category = 5
            elif pill_index < 12:
                category = 4
            elif pill_index < 15:
                category = 0
            elif pill_index < 18:
                category = 1

            # add line for pill to descriptive txt file
            file.write(str(category) + " " + str((x + pillWidth/2)/backWidth) +" " + str((y + pillHeight/2)/backHeight) + " " + str(pillWidth/backWidth) + " " + str(pillHeight/backHeight) + "\n") 
        
    # add random illumination after redering all pills
    enhancer = ImageEnhance.Brightness(trainImage)
    factor = 0.5 + random()
    trainImage = enhancer.enhance(factor)
        
    # save final image
    trainImage = trainImage.resize((1280, 962),Image.ANTIALIAS)
    trainImage.save('TrainingData/' + name + ".jpg", quality=95)
    file.close()

### Create training images without pills

In [7]:
def createEmpty():
    # for each background
    for i in range(0, 58):
        # and each oriatation a image is generated
        for j in range(0, 4):
            name = "TestImage_E" + str(i) + "_"  + str(j)
            trainImage = backgrounds[i].copy()

            trainImage = trainImage.rotate(90*j, expand=True)

            # randomly adjust illumination
            enhancer = ImageEnhance.Brightness(trainImage)
            factor = 0.5 + random()
            trainImage = enhancer.enhance(factor)

            # save image and empty descriptive txt file
            trainImage = trainImage.resize((1280, 962),Image.ANTIALIAS)
            file = open('TrainingData/' + name + ".txt", "w")
            trainImage.save('TrainingData/' + name + ".jpg", quality=95)
            file.close() 

## Generale ALL Data

In [ ]:
for i in range(0, len(pills)):
    for j in range(0, len(backgrounds)):
        for k in range(0, 10):
            #createTestImages(i, j, "TestImage" + str(i) +"_"+ str(j) + "_" + str(k))
            print()
            
for i in range(0, 10000):
    createTestImagesMultPills(i)
    
createEmpty()